# <span style="color:blue">Importing libraries</span>

In [33]:
import pandas as pd
import numpy as np
import re

# <span style="color:blue">Data reading from local files</span>

In [3]:
ufo_by_date = pd.read_csv('ufo_raw_date.csv', index_col=0)

In [4]:
ufo_by_shape = pd.read_csv('ufo_raw_shape.csv', index_col=0)

In [5]:
states = pd.read_csv('states.csv', index_col=0)

# <span style="color:blue">Data cleaning</span>

Number of rows of the dataframes imported from tables arranged by UFO shape and by observation date is the same. Additional column in `ufo_by_date` is the manually introduced `Date_label` column.

In [6]:
ufo_by_date.shape, ufo_by_shape.shape

((135282, 8), (135282, 8))

Datasets collected by shape and by date are identical (result of the first merge is empty), but there are a few duplicates (result of the second merge is longer than the initial dataframe).

In [7]:
pd.merge(left=ufo_by_date, right=ufo_by_shape, how='outer', indicator=True)\
    .query('_merge != "both"').shape

(0, 9)

In [8]:
pd.merge(left=ufo_by_date, right=ufo_by_shape, how='outer', indicator=True)\
    .query('_merge == "both"').shape

(135290, 9)

We will further work with `ufo_raw_date` to keep "manual" date labels.

In [9]:
# making a copy to preserve source data
ufo = ufo_by_date.copy()

In [10]:
ufo

,Date / Time,City,State,Shape,Duration,Summary,Posted,Date_label
0,1/1/10,Kirksville (near),MO,Disk,Minutes,Historical sighting (1903 - 1913) Northern Mis...,9/15/05,01_1910
1,1/28/29 15:41,Santa Teresa,NM,Circle,5 mins,large craft with other smaller crafts moving f...,1/19/21,01_1929
2,1/18/43 07:15,Fiji Islands (S. Pacific Ocean),NaN,NaN,10 sec.,Crew contingents of 2 USN vessels see reconnai...,8/11/04,01_1943
3,1/1/44 12:00,San Diego,CA,Cigar,3 minutes,A sighting of one silver or white enlongated c...,7/25/04,01_1944
4,1/1/44 12:00,Wilderness,WV,Disk,3 weeks,Two related reports of possible predecessor to...,7/8/04,01_1944
...,...,...,...,...,...,...,...,...
135277,NaN,Kelowna (Canada),BC,NaN,NaN,"They were close to my home, hope they come soo...",7/3/13,unspecified
135278,NaN,Rule,TX,Unknown,15 minutes,Unusual sounds and starlight distortion defini...,8/19/12,unspecified
135279,NaN,Gulf Breeze,FL,NaN,NaN,The gulf breeze florida sightings were a AC130...,8/5/12,unspecified
135280,NaN,Richland,WA,Sphere,10 min,Multiple sightings,12/20/12,unspecified


In [11]:
ufo.dtypes

Date / Time    object
City           object
State          object
Shape          object
Duration       object
Summary        object
Posted         object
Date_label     object
dtype: object

We should consider converting `Date / Time` and `Posted` columns to datetime format.
On top of that, the column name with slash and spaces is not convenient.

In [12]:
ufo.rename(columns={'Date / Time': 'Observed'}, inplace=True)

In [18]:
round(sum(ufo.Date_label == 'unspecified') / ufo.shape[0] , 4)

0.0028

Parsing of poorly formatted dates/times is a complicated issue, but these are less than 0.3% of the observations. Hence, the solution is to remove such rows.

Then, we will remove too old observations (say, before 1900). This will resolve the issue of limited range of the `pd.datetime` type; moreover, these observations are not reliable and too scarce to affect the conclusions.

Since year is coded by two last digits, we will extract it from the `Date_label` column. A new `Year` column will be created instead of just query because we will then use it to modify the `Observed` values

In [19]:
%%capture

ufo = ufo.query('Date_label != "unspecified"')

ufo['Year'] = ufo.Date_label.apply(lambda x: int(x[-4:]))

ufo = ufo.query("Year >= 1900")

Without the following insertion of complete year, '68' would be converted to '2068'

In [22]:
times = []

for row in ufo.itertuples():
    date_complete_year = re.sub('/(.. )', '/' + str(row.Year) + ' ', row.Observed)
    times.append(date_complete_year)

ufo['Observed'] = pd.to_datetime(times, format="%m/%d/%Y %H:%M", errors='coerce')

About 1.3% of the dates/times were not complete, even though not marked as 'unspecified'  

An example of such entry is given below (time are missing)

In [23]:
round(ufo.Observed.isnull().sum() / ufo.shape[0], 3)

0.013

In [26]:
pd.DataFrame({
                'as imported': ufo_by_date.loc[125256, :],
                'working': ufo.loc[125256, :]
             })

,as imported,working
City,Meeting Creek (Canada),Meeting Creek (Canada)
Date / Time,12/24/14,NaN
Date_label,12_1914,12_1914
Duration,NaN,NaN
Observed,NaN,NaT
Posted,7/7/17,7/7/17
Shape,NaN,NaN
State,SK,SK
Summary,"I initially submitted this several years ago, ...","I initially submitted this several years ago, ..."
Year,NaN,1914


In the case of `Posted` date, conversion to `datetime` is ambiguous anyway, but we will do it (maybe we will not use this column at all)

In [27]:
ufo.Posted = pd.to_datetime(ufo.Posted, format="%m/%d/%y", errors='coerce')

We don't need `Date_label` and `Year` columns any more.

In [28]:
ufo.drop(columns=['Date_label', 'Year'], inplace=True)

Checking non-null data

There are 49 totally incomplete observations (null data in `Observed`, `City`, `State`, `Shape`, and `Duration` columns). Since we do not aim at analyzing textual data, these observations will be removed.

In [29]:
ufo[ufo[['Observed', 'City', 'Shape', 'State', 'Duration']].isnull().sum(axis=1) == 5].shape[0]

49

In [30]:
ufo = ufo[ufo[['Observed', 'City', 'Shape', 'State', 'Duration']].isnull().sum(axis=1) < 5]

Working with `States` column: we will find the states abbreviations which are not in the `states` dataframe and correct the errors in data input

In [31]:
ufo.query('State not in @states.Abbreviation.values').query('not State.isnull()').State.value_counts()

PQ    102
NF     43
SA     41
YK      6
Fl      5
QB      1
Ca      1
M0      1
Name: State, dtype: int64

* PQ is another assignment of Quebec (QC)
* NF stands for Newfoundland and Labrador (which is NL in `ufo` dataset)
* SA is for Sounth Australia (will keep it in the dataframe as is)
* YK seems to stand for Yukon (YT)
* Fl is misprinted FL (Florida)
* QB seems to be Quebec (QC) as well
* M0 is misprinted MO
* Ca is misprinted CA

In [34]:
new_state = {
                'State': {'PQ': 'QC',
                          'NF': 'NL',
                          'SA': np.nan,
                          'YK': 'YT',
                          'Fl': 'FL',
                          'QB': 'QC',
                          'M0': 'MO',
                          'Ca': 'CA'}
            }

ufo = ufo.replace(new_state)

Now we will analyze the duplicates

5 cases have complete duplicates in the data

In [35]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'], keep=False)]\
    .sort_values(by=['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

,Observed,City,State,Shape,Duration,Summary,Posted
122371,2015-11-15 05:20:00,Reno,NV,Triangle,15 seconds,Black triangle. Only discernible by observing ...,2015-12-10
122372,2015-11-15 05:20:00,Reno,NV,Triangle,15 seconds,Black triangle. Only discernible by observing ...,2015-12-10
25160,2019-03-16 20:45:00,Virginia Beach,VA,Circle,5 minutes,Orange orb. ((anonymous report)),2019-03-21
25161,2019-03-16 20:45:00,Virginia Beach,VA,Circle,5 minutes,Orange orb. ((anonymous report)),2019-03-21
101225,2020-09-14 06:53:00,Aliquippa,PA,NaN,NaN,MADAR Node 68,2020-11-05
101226,2020-09-14 06:53:00,Aliquippa,PA,NaN,NaN,MADAR Node 68,2020-11-05
88646,2021-08-17 22:59:00,Kennett square,PA,Light,15 minutes,"Color changing, circular glowing light spotted...",2021-08-20
88647,2021-08-17 22:59:00,Kennett square,PA,Light,15 minutes,"Color changing, circular glowing light spotted...",2021-08-20
8844,NaT,Grand Junction,CO,NaN,NaN,MADAR Node 75 A number of anomalies this day,2020-01-31
8863,NaT,Grand Junction,CO,NaN,NaN,MADAR Node 75 A number of anomalies this day,2020-01-31


Several more observations differ only in the Posted date

In [36]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary'], keep=False)]\
    .sort_values(by=['Observed', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

,Observed,City,State,Shape,Duration,Summary,Posted
122371,2015-11-15 05:20:00,Reno,NV,Triangle,15 seconds,Black triangle. Only discernible by observing ...,2015-12-10
122372,2015-11-15 05:20:00,Reno,NV,Triangle,15 seconds,Black triangle. Only discernible by observing ...,2015-12-10
86900,2017-08-12 22:30:00,Mount Pleasant,OH,Flash,12 minutes,White flashing light moving irregularly throug...,2017-08-24
86899,2017-08-12 22:30:00,Mount Pleasant,OH,Flash,12 minutes,White flashing light moving irregularly throug...,2017-09-05
8444,2019-01-05 14:00:00,Detroit,MI,Flash,10 minutes,We witnessed 3 bright objects flying in a circ...,2019-01-11
8445,2019-01-05 14:00:00,Detroit,MI,Flash,10 minutes,We witnessed 3 bright objects flying in a circ...,2019-01-17
25160,2019-03-16 20:45:00,Virginia Beach,VA,Circle,5 minutes,Orange orb. ((anonymous report)),2019-03-21
25161,2019-03-16 20:45:00,Virginia Beach,VA,Circle,5 minutes,Orange orb. ((anonymous report)),2019-03-21
113296,2019-10-04 03:00:00,Danville (Canada),QC,Unknown,15 seconds,"2 perfect apples appeared out of nowhere, 1 UF...",2021-07-31
113297,2019-10-04 03:00:00,Danville (Canada),QC,Unknown,15 seconds,"2 perfect apples appeared out of nowhere, 1 UF...",2021-08-16


Quite many observations share place, time, and duration, but Summary is different.

Arbitrary inspection of several first cases has shown that there are minor textual differences in the Summary field. It seems safe to consider them true duplicates.

No more investigation on duplicates on place, time and duration will be performed - let us consider that if at least duration is different, this can indicate multiple objects or multiple (=more reliable) observations of the same object.

In [41]:
ufo[ufo.duplicated(['Observed', 'City', 'State', 'Shape', 'Duration'], keep=False)]\
    .shape

(1192, 7)

Hence, the decision is to remove duplicates basing on `Observed`, `City`, `State`, `Shape`, and `Duration` fields.

This will be done later, upon inspection of the `Shape` column.

In [38]:
ufo.Shape.str.lower().value_counts().sort_index()

changed           1
changing       3283
chevron        1547
cigar          3267
circle        14215
cone            538
crescent          2
cross           433
cylinder       2095
delta             8
diamond        1953
disk           8016
dome              1
egg            1170
fireball       9524
flare             1
flash          2372
formation      4502
hexagon           1
light         27537
other          9272
oval           5859
pyramid           1
rectangle      2302
round             2
sphere         8798
teardrop       1168
triangle      12163
triangular        1
unknown        9218
Name: Shape, dtype: int64

It seems reasonable to 
1) ignore case and  
2) consider (other joinings are possible but questionable):
* 'changed' = 'changing'
* 'cigar' = 'cylinder'
* 'delta' = 'triangular' = 'triangle'
* 'circle' = 'disk' = 'round' = 'sphere'
* 'flare' = 'light'
* 'other' = 'unknown'
* 'egg' = 'oval'
and
3) set NaN to 'unknown'

In [39]:
ufo.Shape = ufo.Shape.str.lower()

new_shape = {
                'Shape': {'changed': 'changing',
                          'cigar': 'cylinder',
                          'delta': 'triangle',
                          'triangular': 'triangle',
                          'disk': 'circle',
                          'round': 'circle',
                          'sphere': 'circle',
                          'flash': 'light',
                          'other': 'unknown',
                          'egg': 'oval',
                          np.nan: 'unknown'}
            }

ufo = ufo.replace(new_shape)

Now we will finally drop the duplicates in view of updated `Shape` column

In [42]:
ufo.drop_duplicates(['Observed', 'City', 'State', 'Shape', 'Duration'], keep='first', inplace=True)

Upon the cleaning, we have kept >99% of the observations.

Many of them (about 12%) have NaN in at least one of the columns, but let us consider these as potentially useful partial information.

In [43]:
round(ufo.shape[0] / ufo_by_date.shape[0], 4)

0.9922

In [44]:
round((ufo.shape[0] - ufo.dropna().shape[0]) / ufo.shape[0], 3)

0.117

In [45]:
# saving cleaned dataframe in case we distort it occasionally during analysis
ufo.to_csv("ufo_for_analysis.csv")